# Creating timeseries of the GOES data for 8 speakers

- 8 channels (need 8 timeseries)
- 12 minute piece, GOES samples every 5-minutes

The artist wanted a viewer to feel like weather was moving through the room. 

This means we must think about time and space.

Space:
So, the 8 speakers should correspond to 8 different locations around the viewer. The spatial scale of the data variablity will need to be explored so we can determine the right distance these locations should be. Maybe if they are only 1 km apart, there isn't enough of a 'difference' for the sound to feel immersive.  But if the locations are too far apart, then they won't be correlated to eachother and may just sound like 8 unconnected sounds.

Time: 
GOES CONUS data has 5-minute sampling. 
The piece is 12 minutes long - so if we went with realtime data, we would have 2 data points that could be interpolated together,
but basically two gradient tones during 12 minutes is not very dynamic, 
but that might be okay? The other issues is that I think this surface data is meant to be dominate during only the first 3 minutes,
so maybe we are only really looking at 3 minutes of gradients. If that is the case, in order to get a sense of weather, 
it might be necessary to create a compressed timeseries of data over the last hour or so.

GOES:

GOES has full disk, CONUS, mesoscale options as well as many different products.
I'm using CONUS below as it has high spatial resolution, 5-minute sampling.

## Steps in code below:
- The code looks at two different GOES datasets. We don't really want one with cloud masks (missing data). 
- Georeference the coordinate system so we can look up data around a latitude and longitude
- Calculate 8 points near a reference location (Miami)
- Create timeseries of data at those points 

In [1]:
from goes2go import GOES
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import s3fs
import xarray as xr
import os

#list of GOES products https://github.com/blaylockbk/goes2go/blob/main/goes2go/product_table.txt

# Calculate latitude and longitude from GOES ABI fixed grid projection data
#GOES ABI fixed grid projection is a map projection relative to the GOES satellite  
#Units: latitude in °N (°S < 0), longitude in °E (°W < 0)  
#See GOES-R Product User Guide (PUG) Volume 5 (L2 products) Section 4.2.8 for details & example of calculations  
#"file_id" is an ABI L1b or L2 .nc file opened using the netCDF4 library  
#code from https://www.star.nesdis.noaa.gov/atmospheric-composition-training/python_abi_lat_lon.php  
#Acknowledgement:  NOAA/NESDIS/STAR Aerosols and Atmospheric Composition Science Team  
#Their code is written for numpy arrays not xarray, so I updated it to work with xarray datasets  

# Target latitude and longitude MIAMI
target_lat = 25.76  # Example: Latitude of Miami
target_lon = -80.19  # Example: Longitude of Miami

#read the dataset so we know what we have already ready or not
gfile = xr.open_dataset("./data/goes_data_time.nc").load()
gfile.close()

def calculate_degrees(file_id):
    # Read in GOES ABI fixed grid projection variables and constants
    x_coordinate_1d = file_id.variables['x'][:]  # E/W scanning angle in radians
    y_coordinate_1d = file_id.variables['y'][:]  # N/S elevation angle in radians
    projection_info = file_id.variables['goes_imager_projection']
    lon_origin = projection_info.attrs.get('longitude_of_projection_origin')
    H = projection_info.attrs.get('perspective_point_height')+projection_info.attrs.get('semi_major_axis')
    r_eq = projection_info.attrs.get('semi_major_axis')
    r_pol = projection_info.attrs.get('semi_minor_axis')
    
    # Create 2D coordinate matrices from 1D coordinate vectors
    x_coordinate_2d, y_coordinate_2d = np.meshgrid(x_coordinate_1d, y_coordinate_1d)
    
    # Equations to calculate latitude and longitude
    lambda_0 = (lon_origin*np.pi)/180.0  
    a_var = np.power(np.sin(x_coordinate_2d),2.0) + (np.power(np.cos(x_coordinate_2d),2.0)*(np.power(np.cos(y_coordinate_2d),2.0)+(((r_eq*r_eq)/(r_pol*r_pol))*np.power(np.sin(y_coordinate_2d),2.0))))
    b_var = -2.0*H*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    c_var = (H**2.0)-(r_eq**2.0)
    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
    s_x = r_s*np.cos(x_coordinate_2d)*np.cos(y_coordinate_2d)
    s_y = - r_s*np.sin(x_coordinate_2d)
    s_z = r_s*np.cos(x_coordinate_2d)*np.sin(y_coordinate_2d)
    
    # Ignore numpy errors for sqrt of negative number; occurs for GOES-16 ABI CONUS sector data
    np.seterr(all='ignore')
    
    abi_lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    abi_lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)
    
    return abi_lat, abi_lon

def forward_fill_2d(arr):
    # Loop through each column
    for i in range(arr.shape[1]):
        mask = np.isnan(arr[:, i])
        # Forward fill NaN values
        arr[mask, i] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), arr[~mask, i])
    return arr

def find_nearest_indices(lat_arr, lon_arr, target_lat, target_lon):
    # Find the nearest latitude index
    lat_idx = (np.abs(lat_arr - target_lat)).argmin()
    # Find the nearest longitude index
    lon_idx = (np.abs(lon_arr - target_lon)).argmin()
    return lat_idx, lon_idx

def calculate_points(istep,lon_idx,lat_idx):
    #how big do we want to have the box?
    #istep is how many grid points away from the center that we want to go   
    # List of points you want to subset around point x
    #   *  *  *
    #   *  x  *
    #   *  *  *
    #north_point = [lat_idx+istep,lon_idx]
    #east_point = [lat_idx,lon_idx+istep]
    #south_point = [lat_idx-istep,lon_idx]
    #west_point = [lat_idx,lon_idx-istep]
    #northeast_point = [lat_idx+istep,lon_idx+istep]
    #northwest_point = [lat_idx+istep,lon_idx-istep]
    #southeast_point = [lat_idx-istep,lon_idx+istep]
    #southwest_point = [lat_idx-istep,lon_idx-istep]
    points = [
        {"i": int(lon_idx)+istep, "j": int(lat_idx)+istep, "name": 'NE'},
        {"i": int(lon_idx)+istep, "j": int(lat_idx), "name": 'East'},
        {"i": int(lon_idx)+istep, "j": int(lat_idx)-istep, "name": 'SW'},
        {"i": int(lon_idx), "j": int(lat_idx)+istep, "name": 'N'},
        {"i": int(lon_idx), "j": int(lat_idx)-istep, "name": 'S'},
        {"i": int(lon_idx)-istep, "j": int(lat_idx)+istep, "name": 'NW'},
        {"i": int(lon_idx)-istep, "j": int(lat_idx), "name": 'W'},
        {"i": int(lon_idx)-istep, "j": int(lat_idx)-istep, "name": 'SW'},
    ]
    return points

def get_start_end_time(fname):
    #goes filenames structure https://geonetcast.wordpress.com/2017/04/27/goes-16-file-naming-convention/
    #use filename to find start/end times for data
    tem = str(fname).split('/')
    tem2,i = tem[5],25
    dt_start = datetime.strptime(tem2[i:i+13], '%Y%j%H%M%S')
    tem2,i = tem[5],41
    dt_end = datetime.strptime(tem2[i:i+13], '%Y%j%H%M%S')
    return dt_start,dt_end

def already_read(fname,gfile):
    start_time,end_time = get_start_end_time(fname)
    isum = gfile.read_value.loc[start_time:end_time].sum().data
    if isum>0:
        return True
    else:
        return False

def set_already_read(fname,gfile):
    start_time,end_time = get_start_end_time(fname)
    gfile.read_value.loc[start_time:end_time] = 1
    gfile.to_netcdf("./data/goes_data_time.nc")
    return gfile


C:\Users\clwhit13\AppData\Local\miniconda3\envs\heart\Lib\site-packages\goes2go\data.py:665: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1h")),
C:\Users\clwhit13\AppData\Local\miniconda3\envs\heart\Lib\site-packages\goes2go\NEW.py:188: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  within=pd.to_timedelta(config["nearesttime"].get("within", "1h")),


# now we have some data points, we need to create a timeseries of data

goes2go downloads the data before reading it  
since we are looking at timeseries and there are like 288 files each day (5 min data)  
i don't want to download all that data  
so i'm trying to figure out if I can lazy load it  


In [6]:
# connect to S3 bucket with data
df = pd.read_csv('./../goes_filenames_test.csv')
df.file[0]
#fs = s3fs.S3FileSystem(anon=True) #connect to s3 bucket!
#file_location=df.file
#file_ob = [fs.open('s3://'+file) for file in file_location]        #open connection to files
#print(file_ob[0])

'noaa-goes16/ABI-L2-MCMIPC/2023/340/00/OR_ABI-L2-MCMIPC-M6_G16_s20233400001173_e20233400003546_c20233400004056.nc'

In [7]:
for i in range(0,len(file_ob)):
    fname = file_ob[i]
#    if already_read(fname,gfile):
#        continue    
    ds = xr.open_dataset(file_ob[i]) #note file is super messed up formatting
    #ds2 = ds#.isel(time=0)
    #calculate lat/lon
    abi_lat, abi_lon = calculate_degrees(ds)
    abi_lat = forward_fill_2d(abi_lat.copy())
    abi_lon = forward_fill_2d(abi_lon.copy())
    #note this isn't going to be perfect because used 1d so run one time, then again with closer values
    # Find nearest indices
    lat_idx, lon_idx = find_nearest_indices(abi_lat[:,0], abi_lon[0,:], target_lat, target_lon)
    lat_idx, lon_idx = find_nearest_indices(abi_lat[:,lon_idx], abi_lon[lat_idx,:], target_lat, target_lon)
    stime,etime = get_start_end_time(fname)
    ds_tem = ds.isel(y=slice(lat_idx-25,lat_idx+25),x=slice(lon_idx-25,lon_idx+25)).CMI_C10.load()
    for istep in range(1,20):
        points = calculate_points(istep,25,25) #lon_idx,lat_idx)
        point_data=ds_tem.isel(y=points[0].get('j'), x=points[0].get('i')) #, method="nearest")
        for p in range(len(points)):
            if p>0:
                tem=ds_tem.isel(y=points[p].get('j'), x=points[p].get('i')) #, method="nearest")
                point_data = xr.concat([point_data, tem], dim="points_index")
        if istep>1:
            step_data = xr.concat([step_data,point_data], dim="step")
        else:
            step_data = point_data
    if i>1:
        all_data = xr.concat([all_data,step_data], dim="time")
    else:
        all_data = step_data
    all_data.to_netcdf(".\..\goes_timeseries_dec2023.nc")
    gfile = set_already_read(fname,gfile)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
file_ob[0]

<File-like object S3FileSystem, noaa-goes16/ABI-L2-MCMIPC/2024/250/18/OR_ABI-L2-MCMIPC-M6_G16_s20242501841171_e20242501843550_c20242501844065.nc>

In [ ]:
#ds.isel(y=points[0].get('j'), x=points[0].get('i')) #, method="nearest" hehe -austin
ds_tem = ds.isel(y=slice(lat_idx-25,lat_idx+25),x=slice(lon_idx-25,lon_idx+25)).CMI_C10.load()
#print(lat_idx, lon_idx)
#print(tem
i=0
if i==0:
    #step_data=ds_tem.isel(y=points[0].get('j'), x=points[0].get('i')) #, method="nearest")
    for istep in range(1,20):
        points = calculate_points(istep,25,25) #lon_idx,lat_idx)
        point_data=ds_tem.isel(y=points[0].get('j'), x=points[0].get('i')) #, method="nearest")
        for p in range(len(points)):
            if p>0:
                tem=ds_tem.isel(y=points[p].get('j'), x=points[p].get('i')) #, method="nearest")
                point_data = xr.concat([point_data, tem], dim="points_index")
        if istep>1:
            step_data = xr.concat([step_data,point_data], dim="step")
        else:
            step_data = point_data
        #subset=combined_data #.load() #now load the little baby dataset
        #save_data = xr.concat([save_data, combined_data], dim="step")
#        save_data.sel(time=np.datetime64(stime),method='nearest',step=istep)["CMI_C10"]=subset.data
        #print(stime,istep,subset,save_data.sel(time=np.datetime64(stime),method='nearest',step=istep)["CMI_C10"])


In [ ]:
step_data

In [ ]:
print(subset.data)

save_data = subset
save_data = xr.concat([save_data,
ds_concat = xr.concat([save_data], dim='time')


save_data.sel(time=np.datetime64(stime),method='nearest',step=0)['CMI_C10']=subset
print(save_data.sel(time=np.datetime64(stime),method='nearest',step=0)["CMI_C10"].data)
#save_data

In [ ]:
print(save_data.CMI_C10.sel(time=np.datetime64(stime),method='nearest',step=0).data)
print(np.datetime64(stime))
#save_data.time[0:40]

In [ ]:
    for istep in range(1,20):
        points = calculate_points(istep,lon_idx,lat_idx)
        # Subsetting the dataset to the points
        combined_data=ds.isel(y=points[0].get('j'), x=points[0].get('i')) #, method="nearest")
        for p in range(len(points)):
            if p>0:
                #print(points[p].get('j'),points[p].get('i'))
                tem=ds.isel(y=points[p].get('j'), x=points[p].get('i')) #, method="nearest")
                combined_data = xr.concat([combined_data, tem], dim="points_index")
        subset=combined_data.CMI_C10.load() #now load the little baby dataset
    stime,etime = get_start_end_time(fname)
    save_data.sel(time=np.datetime64(stime),method='nearest')["CMI_10"]=subset
    save_data.to_netcdf(".\..\goes_timeseries4.nc")
    gfile = set_already_read(fname,gfile)
    print(i)

In [ ]:
        if istep>1:
            subset_step = xr.concat([subset_step,subset],dim='step')
        else:
            subset_step = subset
    if i>1:
        subset_time = xr.concat([subset_time,subset_step],dim='t')
    else:
        subset_time = subset
    subset_time.to_netcdf(".\data\goes_timeseries.nc")
    gfile = set_already_read(fname,gfile)
    print('done :',i)


In [ ]:
save_data

In [ ]:
mylist